# Description

__Goal__

Our goal is to detect interface elements / screens of an app at which users' engagement drops significantly (some users tend to delete their app accounts).


__Tasks__

1. Collect data
2. Prepare data 
3. Analyze data
    1. Build pivot tables
    2. Visualize users' trajectories in the app
    3. Build the classifier
        1. Classifier helps you to pick out specific users' trajectories
        2. Classifier allows you to estimate the probability of user's account being deleted based on user's current trajectory. One can use this information to dynamically change the content of the app to prevent from that.

__Expected results__

1. One will identify the most "problematic" elements of an app
2. One will get the classifier allowing to predict the account deletion based on current user's behaviour

# Download data

In [ ]:
import numpy as np 
import retentioneering
import os
from retentioneering.utils import download_events_multi, preparing
import pandas as pd

In [ ]:
client, job_config = retentioneering.init_from_file('./settings_yaml.yaml')
settings = retentioneering.Config('./settings_yaml.yaml')

#### you can use one settings file in download_events_multi for multiple queries

In [ ]:
# you can see what kind of SQL query is transmitted to BQ
#download_events_multi(client, job_config=job_config, settings=settings, return_only_query=True)[1];
#download_events_multi(client, job_config=job_config, settings=settings, return_only_query=True)[0];

In [ ]:
df = download_events_multi(client, job_config=job_config, settings=settings)
print('Downloaded DataFrame shape: {}'.format(df.shape))

#### create a new column 'group_name' with the name of the query

In [ ]:
directory = '../../data' 
if not os.path.exists(directory):
    os.makedirs(directory)

In [ ]:
path = '../../data/data_from_bq_delete_accounts.csv'
df.to_csv(path, sep=';', encoding='utf-8', index=False)

# Preparing

## Now you can prepare your DataFrame for further analysis using pandas instruments

#### Drop duplicated events

In [ ]:
df = pd.read_csv(path, sep=';', encoding='utf-8')
df = preparing.drop_duplicated_events(df, settings=settings)

#### drop out deleted group users from the test group

In [ ]:
# how many users we have in each group
df.groupby('group_name').user_pseudo_id.nunique()

In [ ]:
# split in two groups
df_deleted = df.loc[df.group_name == u'deleted', :].copy()
df_test = df.loc[df.group_name == u'test_group', :].copy()

# select users from 'deleted' group
selected_users = df_deleted.user_pseudo_id.unique()

# drop selected users from second group
df_test = df_test.loc[~df_test.user_pseudo_id.isin(selected_users)]

print('deleted:', df_deleted.user_pseudo_id.nunique())
print('test_group:', df_test.user_pseudo_id.nunique())

#### leave only users who delete their account in first two days and has 'first_open' event

In [ ]:
# leave only users with first open
has_first_open_users = df_deleted.loc[df_deleted.event_name == u'first_open', 'user_pseudo_id'].unique()
df_deleted = df_deleted.loc[df_deleted.user_pseudo_id.isin(has_first_open_users), :]
print('deleted group users:', df_deleted.user_pseudo_id.nunique())

In [ ]:
# for each user find timestamp when account was deleted, and timestamp of first event
delete_account_time = df_deleted.groupby('user_pseudo_id', as_index=False)['event_timestamp'].max()
first_event_time = df_deleted.groupby('user_pseudo_id', as_index=False)['event_timestamp'].min()

# calculate users lifetime
users_lifetime = first_event_time.merge(delete_account_time, 
                                        how='left', 
                                        on='user_pseudo_id', 
                                        suffixes=('_min', '_del'))

In [ ]:
days_threshold = 2
# select users with timedelta 'days_threshold' days and leave only them
selected_users = users_lifetime[(users_lifetime['event_timestamp_del'] -
                users_lifetime['event_timestamp_min']) / 1e6 / (24*60*60) <= days_threshold].user_pseudo_id.unique()

df_deleted = df_deleted.loc[df_deleted.user_pseudo_id.isin(selected_users), :]
print('deleted group users:', df_deleted.user_pseudo_id.nunique())

In [ ]:
# in second group leave only events within two days
first_event_time = df_test.groupby('user_pseudo_id', as_index=False)['event_timestamp'].min()
df_test = df_test.merge(first_event_time, how='left', on='user_pseudo_id', suffixes=('', '_min'))

In [ ]:
df_test = df_test.loc[(df_test.event_timestamp - df_test.event_timestamp_min) / 1e6 / (24*60*60) <= days_threshold, :]

In [ ]:
# randomly select same size of users from test dataset
np.random.seed(seed=42)
df = df_test.loc[df_test.user_pseudo_id.isin(
    np.random.choice(df_test.user_pseudo_id.unique(), \
                     size=df_deleted.user_pseudo_id.nunique(), \
                     replace=False)), :] \
                    .append(df_deleted, sort=False)

In [ ]:
df.shape

In [ ]:
path = '../../data/data_from_bq_delete_accounts_prepared_data.csv'
df.to_csv(path, sep=';', encoding='utf-8', index=False)

# Analytics

In [ ]:
from retentioneering import analysis, visualization
from retentioneering.utils import preparing

In [ ]:
path = '../../data/data_from_bq_delete_accounts_prepared_data.csv'
df = pd.read_csv(path, sep=';', encoding='utf-8')

In [ ]:
# add accumulator events
df = preparing.add_lost_events(
    df, positive_event_name=u'settings_delete_account_success', negative_event_name=u'not_delete_account')

In [ ]:
# load filters for events
# event_filter - common filters for the problem
# additional_filter - additional filters for the task
# lf - manualy added filters for the task

event_filter = pd.read_csv('NewUserEventList.csv').values
additional_filter = pd.read_csv('additional_filter.csv', ';') 
lf = {'app_provisional_enabledPush', 'app_enabledPush','app_enabledRemotePush'} 
additional_filter = additional_filter[additional_filter.Created_by == '1']['Event Action'].values

In [ ]:
event_filter2 = set(additional_filter)|set(event_filter[:, 0])|{'settings_delete_account_success', 'not_delete_account'}
df = df[df.event_name.isin(event_filter2 - lf)]

In [ ]:
# take only top-30 events for better visualization
top_events = df.groupby('event_name').user_pseudo_id.count()
top_events = set(top_events.sort_values().iloc[-20:].index)|set(['settings_delete_account_success', 'not_delete_account'])
df = df[df.event_name.isin(top_events)]

In [ ]:
# desc = analysis.get_desc_table(df, target_event_list=['settings_delete_account_success',
#                                                       'not_delete_account'],
#                                max_steps=30, settings=settings, plot=True);

In [ ]:
lost_users_list = df[df.event_name == 'settings_delete_account_success'].user_pseudo_id
filt = df.user_pseudo_id.isin(lost_users_list)
df_lost = df[filt]
df_passed = df[~filt]

desc_loss = analysis.get_desc_table(df_lost, target_event_list=['settings_delete_account_success',
                                                                'not_delete_account'],
                                    max_steps=30, settings=settings, plot=False)
desc_passed = analysis.get_desc_table(df_passed, target_event_list=['settings_delete_account_success',
                                                                    'not_delete_account'],
                                      max_steps=30, settings=settings, plot=False)

In [ ]:
diff_df = analysis.get_diff(desc_loss, desc_passed, settings=settings, precalc=True)

Plot graph with python

In [ ]:
df_agg = analysis.get_all_agg(df_lost, agg_list=['trans_count'])
visualization.plot.plot_graph(df_agg, 'trans_count', settings)

Or plot graph via api

`It sends your data on our server`

In [ ]:
from retentioneering.utils.export import plot_graph_api
plot_graph_api(df_lost, settings, task='delete')

Model for prediction delete/non-delete ccount

In [ ]:
# get raw data for model
df = pd.read_csv(path, sep=';', encoding='utf-8')

In [ ]:
# fit model
event_filter_new = (set(additional_filter) | set(event_filter[:, 0]))
#any( item == 'settings_delete_account' for item in event_filter_new)
clf = analysis.Model(df, target_event='settings_delete_account_success', event_filter=event_filter_new,
                     settings=settings)
clf.fit_model()


We can find leaky-events by analysing model output

In [ ]:
importance = clf.model.coef_[0]
names = clf._embedder.inverse_transform([importance])[0]
importance = importance[importance != 0]
tab = list(zip(names, importance))
sorted(tab, key=lambda x: int(x[1]))

We can see that `settings_delete_account` and `settings` has much more importance then other, so we should add it to filter

In [ ]:
# and rebuild model
event_filter_new = (set(additional_filter)|set(event_filter[:, 0])).difference(set(['settings_delete_account', 'settings','profile_settings']))
clf = analysis.Model(df, target_event='settings_delete_account_success',
                     event_filter=event_filter_new,
                     settings=settings)
clf.fit_model()

In [ ]:
# repeat it
importance = clf.model.coef_[0]
names = clf._embedder.inverse_transform([importance])[0]
importance = importance[importance != 0]
list(zip(names, importance))

Now all weights seem to be more uniformly distributed, so we can stop.

We also can get most valued edges and nodes from model to visualize it on graph.

In [ ]:
imp_tracks = clf.build_important_track()
# edges
imp_tracks[imp_tracks[1].notnull()]

In [ ]:
# nodes
imp_tracks[imp_tracks[1].isnull()][0].values